# torchio

In [1]:
import numpy as np
from skimage import io
import napari
import file_utilities as fu

# Get the file path
image_file_path = fu.choose_file()

# Create an instance of LifImageProcessor
# 加载 TIF 格式的图像文件
image = io.imread(image_file_path)
label_file_path = fu.choose_file()

label = io.imread(label_file_path)


Selected File: C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53/bg_remove.tiff
Selected File: C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53/label_wih_border_mask_3.tif


In [2]:
import torchio as tio
import numpy as np
import napari

def apply_transforms_and_save(subject, transforms_list, transpose = False):
    # Create dictionaries to store images and masks
    images_dic = {}
    masks_dic = {}

    transform_names = [transform.__class__.__name__ for transform in transforms_list]
    transform_names = '_'.join(transform_names)

    #print(f'Using {transform_name}....')
    transform = tio.Compose((transforms_list))

    subjects_dataset = tio.SubjectsDataset(subject, transform=transform)
    
    # Get a transformed instance from the dataset
    for i in range(0, len(subjects)):
        transformed_image = subjects_dataset[i]
        transformed_image_keys = transformed_image.keys()
        for key in transformed_image_keys:
            image_type = transformed_image[key].type

                # Extract transformed data
            image_data = transformed_image[key][tio.DATA][i,:,:,:].squeeze().cpu().numpy()

            if image_type == tio.INTENSITY:
                # Extract transformed image data
                image_np = np.swapaxes(image_data, 0, 2)
                if transpose:
                    image_np = np.transpose(image_np, (0, 2, 1))

                images_dic[f'{transform_names} {i} Image'] = image_np

                #print("This is image", key)
            elif image_type == tio.LABEL:
                # Extract transformed mask data
                #print("This is label", key)
                image_mask_np_swapped = np.swapaxes(image_data, 0, 2)
                if transpose:
                    image_mask_np_swapped = np.transpose(image_mask_np_swapped, (0, 2, 1))
                masks_dic[f'{transform_names} {i} Mask'] = image_mask_np_swapped

    # Return the dictionaries
    return images_dic, masks_dic


In [3]:
import torchio as tio
import numpy as np
import napari

def show_transformed_images(images_dic, existing_viewer=None):
    # Check if an existing viewer is provided
    if existing_viewer is None:
        # Create a new napari viewer
        viewer = napari.Viewer()
    else:
        viewer = existing_viewer

    # Add images to the viewer using keys as names
    for key, image in images_dic.items():
        viewer.add_image(image, name=key)

    # Return the viewer
    return viewer

In [4]:
import os
import tifffile
import numpy as np

def save_images_from_dict(image_dict, original_image_path, suffix = None):
    # Check if the dictionary keys contain 'image' or 'mask', if not raise an error
    valid_keys = [key for key in image_dict.keys() if 'image' in key.lower() or 'mask' in key.lower()]
    if len(valid_keys) != len(image_dict):
        raise ValueError("Dictionary keys must contain 'image' or 'mask'")

    # Get the directory path of the file
    file_directory = os.path.dirname(original_image_path)
    
    # Get the name of the last folder
    last_folder_name = os.path.basename(file_directory)
    
    # The target directory is the 'Augmentation' folder in the current directory
    target_directory = os.path.join(os.getcwd(), "Augmentation")
    
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    
    # Create a subdirectory with the same name as the last folder in the target directory
    last_folder_path = os.path.join(target_directory, last_folder_name)
    if not os.path.exists(last_folder_path):
        os.makedirs(last_folder_path)

    for key, image in image_dict.items():
        # Determine the name of the subfolder for storing images or masks
        subfolder_name = 'image' if 'image' in key.lower() else 'mask'
        
        # Create the path for the subfolder
        subfolder_path = os.path.join(last_folder_path, subfolder_name)
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Split the key to extract the image number
        parts = key.split()
        image_number = parts[-2]
        
        # Determine if it's an image or mask
        if 'image' in key.lower():
            file_type = 'Image'
        elif 'mask' in key.lower():
            file_type = 'Mask'
        else:
            raise ValueError(f"Invalid key: {key}")
        
        # Construct the file path with suffix
        if suffix:
            file_path = os.path.join(subfolder_path, f"{'_'.join(parts[:-2])}_{suffix} {image_number} {file_type}.tif")
        else:
            file_path = os.path.join(subfolder_path, f"{key}.tif")
            
        # Save the image
        if isinstance(image, np.ndarray):
            tifffile.imwrite(file_path, image)
            #print(f"Saved {key} to {file_path}")
        else:
            raise ValueError(f"Image data associated with key '{key}' is not a numpy array")
            
        #print(f"Saved {key} to {file_path}")

In [5]:
import itertools

def compose_transforms(transforms_list, Max_num_transforms = None):
    # Get all combinations of transforms_list with length up to Max_num_transforms
    if Max_num_transforms==None:
        Max_num_transforms = len(transforms_list)
    all_combinations = []
    for i in range(1, Max_num_transforms + 1):
        combinations = list(itertools.combinations(transforms_list, i))
        all_combinations.extend(combinations)
    #for i, combination in enumerate(all_combinations):
    #    print(f"Combination {i+1}: {combination}")
    print(f"You have {len(all_combinations)} combinations.")

    return all_combinations

In [7]:
subjects = [
    tio.Subject(
        t1=tio.ScalarImage(image_file_path),
        label=tio.LabelMap(label_file_path),
    )
]

transforms_list = [
    #tio.transforms.RandomBlur((2,3)),
    #tio.transforms.RandomNoise(10, 2),
    #tio.transforms.RandomBiasField(),
    tio.transforms.RandomGamma(),
    tio.RandomFlip(axes=('LR'), flip_probability = 1), 
    #tio.RandomAffine(scales= (1,1), degrees = (15)), 
    #tio.RandomElasticDeformation(num_control_points = (7,7,5), max_displacement = (7,7,4)), 
    #tio.RandomAnisotropy()
]
transforms_combinations = compose_transforms(transforms_list)

for i, combination in enumerate(transforms_combinations):
    images, masks = apply_transforms_and_save(subjects, combination, transpose = True)
    save_images_from_dict(images, image_file_path, 'LR transpose')
    save_images_from_dict(masks, label_file_path, 'LR transpose')

You have 3 combinations.


In [80]:
viewer = show_transformed_images(images, viewer)